In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import regex as re
from values import *

In [ ]:
val = Values()

In [ ]:
df = pd.read_csv(val.file_path, encoding='latin-1', delimiter=';', on_bad_lines='skip', parse_dates= val.dates, dayfirst=True)


In [ ]:
kuchen_gerate = df[df['WARENGR'].isin(val.wr_list)]

In [ ]:
for id,item in enumerate(kuchen_gerate['VAR_TEXT']):
    kuchen_gerate = kuchen_gerate.dropna(subset='VAR_TEXT')


### Cleaning unwanted characters and rows

In [ ]:
kuchen_gerate['VAR_TEXT'] = kuchen_gerate['VAR_TEXT'].str.replace('&nbsp_',' ')
kuchen_gerate['VAR_TEXT'] =kuchen_gerate['VAR_TEXT'].str.replace('&Oslash',' ')
kuchen_gerate['VAR_TEXT'] =kuchen_gerate['VAR_TEXT'].str.replace('<br>',' ')

kuchen_gerate = kuchen_gerate[kuchen_gerate['WM'].isna() == True]
kuchen_gerate = kuchen_gerate[kuchen_gerate['NUMMER'].str.match(r'^\w+S$') == False]
kuchen_gerate = kuchen_gerate[kuchen_gerate['NUMMER'].str.match(r'^\w+P$') == False]


In [ ]:
mined_text = kuchen_gerate[['NUMMER','VAR_TEXT']].copy()

In [ ]:
mined_text.loc[mined_text['VAR_TEXT'].str.contains(' klein *',case=False),'GROESSE'] = 'S'
mined_text.loc[mined_text['VAR_TEXT'].str.contains(' groß *',case=False),'GROESSE'] = 'L'
mined_text.loc[mined_text['VAR_TEXT'].str.contains(' mittel *',case=False),'GROESSE'] = 'M'


mined_text['FARBE'] = mined_text['VAR_TEXT'].str.extract(r'(\Wrot\W|kirschrot|ofenrot|orange|gelb|blau|schwarz|weiß|weiss|grün|silber|creme|grau|blu |dijongelb|Elfenbein|gold|aubergine|Dunkelgrün|dunkel- braun|braun)',flags=re.IGNORECASE)
mined_text['FARBE'] = mined_text['FARBE'].str.capitalize()

mined_text['STUECK'] = mined_text['VAR_TEXT'].str.extract(r'( \d+ Stück| *\d+-tlg.|\d+-teilig|\d+tlg.)',flags=re.IGNORECASE)
# mined_text['MATERIAL'] = mined_text['VAR_TEXT'].str.extract(r'(glas|edelstahl|ohne keramik|Cristal|Guss-Aluminium|titan|kupfer|edelst.|keramik|Gußeisen|Gusseisen|Guss|eisen|holz|porzellan)',flags=re.IGNORECASE)
# mined_text['MATERIAL'] = mined_text['MATERIAL'].str.capitalize()
mined_text['GEWICHT'] = mined_text['VAR_TEXT'].str.extract(r'([\d,]*\d+ *kg|\d+ *g |\d+ *gram)',flags=re.IGNORECASE)
mined_text['HOHE'] = mined_text['VAR_TEXT'].str.extract(r'(\d+ *cm hoch|Pfanne hoch \(\d+ cm\)|[\d,\d]+ cm hoch)',flags=re.IGNORECASE)
mined_text['BODEN'] = mined_text['VAR_TEXT'].str.extract(r'(Boden [Ø ]*\d+ cm)',flags=re.IGNORECASE)
mined_text['DECKEL'] = mined_text['VAR_TEXT'].str.extract(r'(mit deckel|ohne deckel|mit glasdeckel|m.Deckel|\+Deckel|\+ Deckel)',flags=re.IGNORECASE)
mined_text['FORM'] = mined_text['VAR_TEXT'].str.extract(r'(rund|oval|rechteckige|eckige|rechteckig|eckig)',flags=re.IGNORECASE)
mined_text['FORM'] = mined_text['FORM'].str.capitalize()



### Separating dimensions into separate columns

In [ ]:
mined_text['MASSEN'] = mined_text['VAR_TEXT'].str.extract(r'([\d,*\d* x]*[ *\d,*\d* x]*[ *\d,*\d*] *cm| \d+mm| \d+ mm| \d+cm| \d+mm|\d+ cm)',flags=re.IGNORECASE)

mined_text['MASSEN'] = mined_text['MASSEN'].str.replace(r'^, ','',regex=True)
mined_text['MASSEN'] = mined_text['MASSEN'].str.replace(r'^ *','',regex=True)
mined_text['MASSEN'] = mined_text['MASSEN'].str.replace('X','x',regex=True)
mined_text['MASSEN'] = mined_text['MASSEN'].str.replace('x',' x ',regex=True)
mined_text['MASSEN'] = mined_text['MASSEN'].str.replace('  x  ',' x ',regex=True)
mined_text['MASSEN'] = mined_text['MASSEN'].str.replace('cm',' cm',regex=True)
mined_text['MASSEN'] = mined_text['MASSEN'].str.replace('  cm',' cm',regex=True)
mined_text['MASSEN'] = mined_text['MASSEN'].str.replace('mm',' mm',regex=True)

mined_text['MASSEN_EINHEIT'] = mined_text['MASSEN'].str.extract(r'(cm|mm)')
mined_text['MASSEN'] = mined_text['MASSEN'].str.replace(r'[cm|mm]','',regex=True)

### Cleaning up the values in columns (making them uniform)

In [ ]:
mined_text['VOLUME'] = mined_text['VAR_TEXT'].str.extract(r'( *\d+ ml| *\d+l| *\d+,\d+ *l| *\d+ l)',flags=re.IGNORECASE)
mined_text['VOLUME'] = mined_text['VOLUME'].str.replace(r'L',' L',regex=True)
mined_text['VOLUME'] = mined_text['VOLUME'].str.replace(r'l',' L',regex=True)


mined_text['VOLUME'] = mined_text['VOLUME'].str.replace(r'  L',' L',regex=True)
mined_text['VOLUME'] = mined_text['VOLUME'].str.replace(r'm L','mL',regex=True)

mined_text['VOLUME_EINHEIT'] = mined_text['VOLUME'].str.extract(r'(L|mL)')
mined_text['VOLUME'] = mined_text['VOLUME'].str.replace(r'[L|mL]','',regex=True)

In [ ]:
mined_text['HOHE'] = mined_text['HOHE'].str.replace('cm',' cm')
mined_text['HOHE'] = mined_text['HOHE'].str.replace('  cm',' cm')


In [ ]:
mined_text['BODEN'] = mined_text['BODEN'].str.replace(' Ø','')
mined_text['BODEN'] = mined_text['BODEN'].str.replace(' ø','')

In [ ]:
mined_text['STUECK'] = mined_text['STUECK'].replace(r'-tlg.|-tlg |\dtlg.|-teilig|\dtlg ',' tlg.',regex=True)
mined_text['STUECK'] = mined_text['STUECK'].lstrip()

In [ ]:
text = mined_text['MASSEN'].str.split(r'x|, ')

df = pd.DataFrame(text)

df['MASSEN_1'] = ''
df['MASSEN_2'] = ''
df['MASSEN_3'] = ''

for index, row in df.iterrows():
    numbers = row['MASSEN']
    if isinstance(numbers, list):
        if len(numbers) == 1:
            df.at[index, 'MASSEN_1'] = numbers[0]
        elif len(numbers) == 2:
            df.at[index, 'MASSEN_1'] = numbers[0]
            df.at[index, 'MASSEN_2'] = numbers[1]
        elif len(numbers) == 3:
            df.at[index, 'MASSEN_1'] = numbers[0]
            df.at[index, 'MASSEN_2'] = numbers[1]
            df.at[index, 'MASSEN_3'] = numbers[2]

df.drop(columns=['MASSEN'], inplace=True)

mined_text['MASSEN_1'] = df['MASSEN_1']
mined_text['MASSEN_2'] = df['MASSEN_2']
mined_text['MASSEN_3'] = df['MASSEN_3']

In [ ]:
new_data = mined_text['VAR_TEXT'].str.extractall(r'(edelstahl|ohne keramik|Cristal|Guss-Aluminium|Gussaluminium|Anti- haftbeschichtung|titan|kupfer|edelst.|keramik|Gußeisen|Gusseisen|Guss|eisen|holz|porzellan)',flags=re.IGNORECASE).groupby(level=0).agg(','.join)

df_l = pd.DataFrame(new_data)
new_df = df_l[0].str.split(',')
# new_df = new_df.str.capitalize()

In [ ]:
df = pd.DataFrame(new_df)

df['MATERIAL_1'] = ''
df['MATERIAL_2'] = ''
df['MATERIAL_3'] = ''


for index, row in df.iterrows():
    numbers = row[0]
    if isinstance(numbers, list):
        if len(numbers) == 1:
            df.at[index, 'MATERIAL_1'] = numbers[0]
        elif len(numbers) == 2:
            df.at[index, 'MATERIAL_1'] = numbers[0]
            df.at[index, 'MATERIAL_2'] = numbers[1]
        elif len(numbers) == 3:
            df.at[index, 'MATERIAL_1'] = numbers[0]
            df.at[index, 'MATERIAL_2'] = numbers[1]
            df.at[index, 'MATERIAL_3'] = numbers[2]



In [ ]:
mined_text['MATERIAL_1'] = df['MATERIAL_1']
mined_text['MATERIAL_2'] = df['MATERIAL_2']
mined_text['MATERIAL_3'] = df['MATERIAL_3']
# mined_text.drop(columns=['MATERIAL'], inplace=True)

mined_text['MATERIAL_1'] = mined_text['MATERIAL_1'].str.capitalize()
mined_text['MATERIAL_2'] = mined_text['MATERIAL_2'].str.capitalize()
mined_text['MATERIAL_3'] = mined_text['MATERIAL_3'].str.capitalize()


In [ ]:
mined_text = mined_text[['NUMMER', 'VAR_TEXT', 'GROESSE', 'FARBE', 'VOLUME','VOLUME_EINHEIT', 'MASSEN_1','MASSEN_2','MASSEN_3', 'MASSEN_EINHEIT', 'STUECK',
      'GEWICHT', 'HOHE', 'BODEN', 'DECKEL', 'FORM','MATERIAL_1','MATERIAL_2','MATERIAL_3']]

In [ ]:
mined_text.to_excel('kuchen_gerate_mined_sample.xlsx')